In [ ]:
import sys, importlib, time
sys.path.append('../../')

from src.Modules.Utils.Imports import *
import src.Modules.Loaders.DataFormatter as DF

from src.DE_simulation import DE_sim, simple_adhesion_diffusion, simple_pulling_diffusion
from src.get_params import get_pulling_params, get_adhesion_params, get_heterog_params
from src.custom_functions import to_torch, to_numpy, load_model, recover_binn_params, unique_inputs

### Simulate MF Models

In [ ]:
'''
Simulate and time the mean-field PDEs for the Pulling, Adhesion, and 
Pulling & Adhesion ABMs

scenario = "pulling" for the Pulling ABM
scenario = "adhesion" for the Adhesion ABM
scenario = "adhesion_pulling" for the Pulling & Adhesion ABM
'''

scenario = "adhesion_pulling"
assert scenario in ["adhesion","pulling","adhesion_pulling"]

data_path = "../../data/"

if scenario == "pulling":
    filename_header = "simple_pulling_mean_25"
    diffusion = simple_pulling_diffusion
    param_combos = get_pulling_params()
    int_str = "Ppull"
    PDE_type = "one-compartment"
elif scenario == "adhesion":
    filename_header = "simple_adhesion_mean_25"
    diffusion = simple_adhesion_diffusion
    param_combos = get_adhesion_params()
    int_str = "Padh"
    PDE_type = "one-compartment"   
elif scenario == "adhesion_pulling":
    filename_header = "adhesion_pulling_mean_25"
    diffusion = []
    param_combos = get_heterog_params()
    PDE_type = "two-compartment"
    
for params in param_combos:
    
    ### Simulate Mean-field model
    if scenario == "adhesion_pulling":
        PmH, PmP, Padh, Ppull, alpha = params
        file_name = f'{filename_header}_PmH_{PmH}_PmP_{PmP}_Padh_{Padh}_Ppull_{Ppull}_alpha_{alpha}'
    else:
        Pm, Pint = params
        file_name = f'{filename_header}_Pm_{Pm}_{int_str}_{Pint}'
    inputs, outputs, shape  = DF.load_ABM_data(data_path+file_name+".npy",plot=False)

    if scenario == "adhesion" and Pint > 0.75:
        pass
    else:
        x,t = unique_inputs(inputs)
        U = outputs.reshape((len(x),-1))
        
        if scenario == "adhesion_pulling":
            
            q = [PmH/4, PmP/4, Padh, Ppull]
            
            IC_mat = np.load(data_path+file_name+".npy",allow_pickle=True).item()
            compartmental_data = IC_mat['compartments']
            Hcomp, Pcomp = compartmental_data[0,:,:], compartmental_data[1,:,:]
            IC = np.hstack( (Hcomp[:,0], Pcomp[:,0]) )

        else:
            q = [Pm/4,Pint]
            IC = U[:,0]

        t0 = time.time()
        sol = DE_sim(x, 
                    t, 
                    q, 
                    IC, 
                    Diffusion_function = diffusion,
                    PDE_type = PDE_type)
        cpu_time = time.time() - t0

        if scenario == "adhesion_pulling":
            filename_head_params = f'{filename_header}_PmH_{PmH}_PmP_{PmP}_Padh_{Padh}_Ppull_{Ppull}_alpha_{alpha}'
        else:
            filename_head_params = f'{filename_header}_Pm_{Pm}_{int_str}_{Pint}'
        save_filename = f"../../results/timing/MF_PDE_sim_{filename_head_params}"

        data = {}
        data['time'] = cpu_time
        np.save(save_filename,data)